<a href="https://colab.research.google.com/github/harshitEbPandey/transferLearningNER/blob/mlp_models/Malu_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
from datasets import load_dataset

In [ ]:
!pip install fasttext

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import to_categorical
from keras import optimizers
from keras import regularizers
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import keras
# import fasttext
from tensorflow import random
from sklearn.metrics import f1_score, classification_report, ConfusionMatrixDisplay

### Tokenizing and saving txt (X_train_ml, Y_train_ml)

In [ ]:
from datasets import load_dataset
ml_ner = load_dataset('ai4bharat/naamapadam', 'ml')

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset naamapadam downloaded and prepared to /root/.cache/huggingface/datasets/ai4bharat___naamapadam/ml/1.0.0/c1b045180d60b208d2468bdad897d04461f08c7137c04a85220697b1bef7df9a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# analysing the hindi dataset
ml_ds = ml_ner.map()

# shapes of each
print(f"Train: {ml_ds['train'].shape}")
print(f"Test: {ml_ds['test'].shape}")
print(f"Validation: {ml_ds['validation'].shape}")

# loading each
train_ml_ds = ml_ds['train'].to_pandas()
test_ml_ds = ml_ds['test'].to_pandas()
vali_ml_ds = ml_ds['validation'].to_pandas()

Map:   0%|          | 0/716652 [00:00<?, ? examples/s]

Map:   0%|          | 0/974 [00:00<?, ? examples/s]

Map:   0%|          | 0/3618 [00:00<?, ? examples/s]

Train: (716652, 2)
Test: (974, 2)
Validation: (3618, 2)


In [ ]:
file_loc = '/content/drive/MyDrive/CSE 572/Datasets/Malayalam'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# combine each of the words in the individual sentences and create a common df
# text output is of the form in X_train.txt: `tokens`
# text output is of the form in Y_train.txt: `ner_tags`
import pandas as pd
with open(file_loc + '/X_train_ml_ds.txt', 'w') as f1, open(file_loc + '/Y_train_ml_ds.txt', 'w') as f2:
  for index, row in train_ml_ds.iterrows():
      f1.write('\n'.join(row['tokens']) + '\n')
      f2.write('\n'.join(map(str, row['ner_tags'])) + '\n')

### Loading the tokens and tags from txt

In [ ]:
# all tokens and tags are presaved...
# taking them input and making unique dictionary
# Open the input file and read the words
with open(file_loc + '/X_train_ml_ds.txt', 'r') as f:
    words = f.read().splitlines()

In [ ]:
with open(file_loc + '/Y_train_ml_ds.txt', 'r') as f:
    tags = f.read().splitlines()

In [ ]:
len(tags) == len(words)

True

In [ ]:
word_tag_unique = {}
for idx, word in enumerate(words):
  if word not in word_tag_unique:
    word_tag_unique[word] = [tags[idx]]
  elif word in word_tag_unique and tags[idx] not in word_tag_unique[word]:
    word_tag_unique[word].append(tags[idx])

In [ ]:
with open(file_loc + '/train_ml_unique_tokens.txt', 'w') as f:
  for word in word_tag_unique.keys():
    f.write(word + '\n')

### Training fasttext on unique words dimensions = 30

In [ ]:
# run fasttext model
model = fasttext.train_unsupervised(file_loc + '/train_ml_unique_tokens.txt', model='skipgram', lr=0.05, dim=30, ws=5, epoch=5)

In [ ]:
# save the model
model.save_model(file_loc + '/fasttext_model_train_ml_unique_tokens.bin')

In [ ]:
# load the model
model = fasttext.load_model(file_loc + '/fasttext_model_train_ml_unique_tokens.bin')

In [ ]:
# every word should have an embedding now
# can get using model[word]

### Loading the training set

In [ ]:
# constructing x_train_df

word_dict = {i: word for i, word in enumerate(words)}
X_train_df = pd.DataFrame.from_dict(word_dict, orient='index', columns=['tokens'])

In [ ]:
# constructing y_train_df

tag_dict = {i: tag for i, tag in enumerate(tags)}
Y_train_df = pd.DataFrame.from_dict(tag_dict, orient='index', columns=['tags'])

### Saving word vectors in chunks

In [ ]:
def process_chunk(start_idx, end_idx, df):
    # Get the words for the chunk
    words = df.loc[start_idx:end_idx, 'tokens'].tolist()

   # Get the vectors for the words using the fastText model
    vectors = [model[word] for word in words]

    # list of index
    idx = range(start_idx, end_idx + 1)

    # Create a new dataframe with the vectors
    vectors_df = pd.DataFrame({'tokens': words, 'vectors': vectors}, index = idx)

    # # Save the chunk to a CSV file
    vectors_df.to_csv(file_loc + f'/vectors/ml_lite_vectors_{start_idx}_{end_idx}.csv', index=False)

In [ ]:
X_train_df.shape

(6580163, 1)

In [ ]:
# Loop over the dataframe in chunks
chunk_size = 1000000
for i in range(0, len(X_train_df), chunk_size):
    start_idx = i
    end_idx = min(i + chunk_size, len(X_train_df)-1)
    process_chunk(start_idx, end_idx, X_train_df)
    # Print progress
    print(f'Saved vectors for words {start_idx} to {end_idx} to CSV file.')

Saved vectors for words 0 to 1000000 to CSV file.
Saved vectors for words 1000000 to 2000000 to CSV file.
Saved vectors for words 2000000 to 3000000 to CSV file.
Saved vectors for words 3000000 to 4000000 to CSV file.
Saved vectors for words 4000000 to 5000000 to CSV file.
Saved vectors for words 5000000 to 6000000 to CSV file.
Saved vectors for words 6000000 to 6580162 to CSV file.


In [ ]:
# encode the labels
label_encoder = LabelEncoder()
label_encoder.fit(Y_train_df.tags.values)
Y_train_ml_enc = label_encoder.transform(Y_train_df.tags.values)

In [ ]:
# get each class' weights
unique, counts = np.unique(Y_train_ml_enc, return_counts=True)
class_weights = dict(zip(unique, np.round(sum(counts) / counts)))

# more the number of records - lesser the weights are (for balancing dataset)
print(class_weights)

{0: 1.0, 1: 13.0, 2: 16.0, 3: 28.0, 4: 35.0, 5: 21.0, 6: 112.0}


In [ ]:
Y_train_ml_enc.shape

(6580163,)

### Training MLP CLassifier

In [ ]:
np.unique(Y_train_ml_enc)

array([0, 1, 2, 3, 4, 5, 6])

In [ ]:
num_features = 30
print(num_features)
num_classes = 7

30


In [ ]:
# Set the random seed
np.random.seed(42)
random.set_seed(42)

mlp = Sequential()
mlp.add(Dense(units=100, activation='relu', input_dim=num_features))
mlp.add(Dense(units=num_classes, activation='softmax'))
mlp.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Save Test data

In [ ]:
with open(file_loc + '/X_test_ml_ds.txt', 'w') as f1, open(file_loc + '/Y_test_ml_ds.txt', 'w') as f2:
  for index, row in test_ml_ds.iterrows():
      f1.write('\n'.join(row['tokens']) + '\n')
      f2.write('\n'.join(map(str, row['ner_tags'])) + '\n')

### Load test data

In [ ]:
# Load the pre-trained fastText model
model = fasttext.load_model(file_loc + '/fasttext_model_train_ml_unique_tokens.bin')

# Open the input file and read the words
with open(file_loc + '/X_test_ml_ds.txt', 'r') as f:
    words = [line.strip() for line in f if line.strip()]

# Create a dictionary with the words as values and integer indices as keys
word_dict = {i: word for i, word in enumerate(words)}

# Create a dataframe from the dictionary
df_test = pd.DataFrame.from_dict(word_dict, orient='index', columns=['tokens'])

In [ ]:
# Loop over the dataframe in chunks
start_idx = 0
end_idx = len(df_test) - 1
process_chunk(start_idx, end_idx, df_test)
# Print progress
print(f'Saved test vectors for words {start_idx} to {end_idx} to CSV file.')

# get the labels in another dataframe
with open(file_loc +'/Y_test_ml_ds.txt', 'r') as f:
    test_tags = [line.strip() for line in f if line.strip()]

tag_dict = {i: tag for i, tag in enumerate(test_tags)}

df_test_tags = pd.DataFrame.from_dict(tag_dict, orient='index', columns=['tags'])

X_test = pd.read_csv(file_loc + f'/vectors/ml_lite_vectors_{start_idx}_{end_idx}.csv')
X_test.head()

Saved test vectors for words 0 to 8533 to CSV file.


,tokens,vectors
0,കോൺഗ്രസ്,[ 0.00033858 -0.00063173 0.00070584 -0.002056...
1,സംസ്ഥാന,[-1.3820477e-03 -6.1588257e-04 4.7179923e-04 ...
2,അധ്യക്ഷൻ,[ 1.2214375e-03 -3.9169376e-04 5.7172740e-04 ...
3,ജി,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
4,പരമേശ്വര,[ 1.2189770e-03 -1.7993987e-04 5.8963854e-04 ...


In [ ]:
# preparing data for prediction

target_names = ['Class 0', 'Class 1', 'Class 2', 'Class 3', 'Class 4', 'Class 5', 'Class 6']

y_test_ml_ds = label_encoder.transform(df_test_tags.tags.values)

X_test = np.array([np.fromstring(s[1:-1].replace('\n', ' '), sep=' ') for s in X_test.vectors])

Y_test = y_test_ml_ds

y_true_labels = Y_test

In [ ]:
# training model pipeline (in chunks)

# Load data in chunks, and train
# training sets
# Loop over the dataframe in chunks
chunk_size = 1000000
for i in range(0, len(X_train_df), chunk_size):
    start_idx = i
    end_idx = min(i + chunk_size, len(X_train_df)-1)
    X_train = pd.read_csv(file_loc + f'/vectors/ml_lite_vectors_{start_idx}_{end_idx}.csv')
    X_train = np.array([np.fromstring(s[1:-1].replace('\n', ' '), sep=' ') for s in X_train.vectors])
    Y_train = Y_train_ml_enc[start_idx: end_idx + 1]

    # Print progress
    print(f'Loaded vectors for words {start_idx} to {end_idx} to CSV file.')
    
    # train mlp on this
    mlp.fit(X_train, Y_train, class_weight=class_weights, epochs = 50, batch_size = 200)

    # predict on test
    Y_test_pred = mlp.predict(X_test)

    # get pred labels
    y_pred_labels = np.argmax(Y_test_pred, axis=1)

    # print classification report
    print(classification_report(y_true_labels, y_pred_labels, target_names=target_names))

    # save model
    mlp.save(file_loc + f'/models/mlp_model_ml_lite_50_epochs.h5')

    # Print progress
    print(f'Completed training with vectors for words {start_idx} to {end_idx}')

## ML-ML Baseline -- 50 epochs
```
         precision    recall  f1-score   support

     Class 0       0.83      0.54      0.65      6052
     Class 1       0.36      0.39      0.37       714
     Class 2       0.30      0.24      0.27       629
     Class 3       0.15      0.45      0.22       309
     Class 4       0.08      0.23      0.12       249
     Class 5       0.25      0.60      0.36       482
     Class 6       0.07      0.40      0.11        99

    accuracy                           0.49      8534
   macro avg       0.29      0.41      0.30      8534
weighted avg       0.67      0.49      0.55      8534
```
Completed training with vectors for all training words 6580162

In [ ]:
def run_model(epoch):
  chunk_size = 1000000
  for i in range(0, len(X_train_df), chunk_size):
      start_idx = i
      end_idx = min(i + chunk_size, len(X_train_df)-1)
      X_train = pd.read_csv(file_loc + f'/vectors/ml_lite_vectors_{start_idx}_{end_idx}.csv')
      X_train = np.array([np.fromstring(s[1:-1].replace('\n', ' '), sep=' ') for s in X_train.vectors])
      Y_train = Y_train_ml_enc[start_idx: end_idx + 1]

      # Print progress
      print(f'Loaded vectors for words {start_idx} to {end_idx} to CSV file.')
      
      # train mlp on this
      mlp.fit(X_train, Y_train, class_weight=class_weights, epochs = epoch, batch_size = 200)

      # predict on test
      Y_test_pred = mlp.predict(X_test)

      # get pred labels
      y_pred_labels = np.argmax(Y_test_pred, axis=1)

      # print classification report
      print(classification_report(y_true_labels, y_pred_labels, target_names=target_names))

      # save model
      mlp.save(file_loc + f'/models/mlp_model_ml_lite_{epoch}_epochs.h5')

      # Print progress
      print(f'Completed training with vectors for words {start_idx} to {end_idx}')

In [ ]:
run_model(100)

### Same as above but 100 epochs
```
           precision    recall  f1-score   support

     Class 0       0.95      0.17      0.29      6052
     Class 1       0.28      0.39      0.33       714
     Class 2       0.11      0.66      0.19       629
     Class 3       0.27      0.39      0.32       309
     Class 4       0.12      0.20      0.15       249
     Class 5       0.27      0.59      0.37       482
     Class 6       0.06      0.45      0.10        99

    accuracy                           0.26      8534
   macro avg       0.30      0.41      0.25      8534
weighted avg       0.74      0.26      0.28      8534

```

In [ ]:
file_loc = '/content/drive/MyDrive/CSE 572/Datasets/Malayalam'

In [ ]:
# get the labels in another dataframe
with open(file_loc +'/Y_test_ml_ds.txt', 'r') as f:
    test_tags = [line.strip() for line in f if line.strip()]
tag_dict = {i: tag for i, tag in enumerate(test_tags)}

df_test_tags = pd.DataFrame.from_dict(tag_dict, orient='index', columns=['tags'])

# X_test = pd.read_csv(file_loc + f'/vectors/te_lite_vectors_{start_idx}_{end_idx}.csv')
# X_test.head()
df_test_tags.tags.values

array(['3', '0', '0', ..., '0', '0', '0'], dtype=object)

In [ ]:
X_test = pd.read_csv(file_loc + f'/vectors/ml_lite_vectors_0_8533.csv')

# preparing data for prediction

target_names = ['Class 0', 'Class 1', 'Class 2', 'Class 3', 'Class 4', 'Class 5', 'Class 6']

label_encoder = LabelEncoder()
label_encoder.fit(df_test_tags.tags.values)
Y_test_te_ds = label_encoder.transform(df_test_tags.tags.values)

X_test = np.array([np.fromstring(s[1:-1].replace('\n', ' '), sep=' ') for s in X_test.vectors])

Y_test = Y_test_te_ds

y_true_labels = Y_test

In [ ]:
mlp = keras.models.load_model(file_loc + '/models/mlp_model_ml_lite_50_epochs.h5')

# predict on test
Y_test_pred = mlp.predict(X_test)

# get pred labels
y_pred_labels = np.argmax(Y_test_pred, axis=1)

# print classification report
print(classification_report(y_true_labels, y_pred_labels, target_names=target_names))

267/267 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

     Class 0       0.83      0.54      0.65      6052
     Class 1       0.36      0.39      0.37       714
     Class 2       0.30      0.24      0.27       629
     Class 3       0.15      0.45      0.22       309
     Class 4       0.08      0.23      0.12       249
     Class 5       0.25      0.60      0.36       482
     Class 6       0.07      0.40      0.11        99

    accuracy                           0.49      8534
   macro avg       0.29      0.41      0.30      8534
weighted avg       0.67      0.49      0.55      8534



In [ ]:
def eval_results_hmm(y_test, y_pred, clf_report=True, conf_matrix=False, exclude_0=True):
    """
        Report evaluation metrics for the NER model (HMM)
    """
    labels = np.arange(1,7) if exclude_0 else np.arange(0,7)
    print("Weighted F1 of HMM = {:.4f}".format(f1_score(y_test, y_pred, average='weighted', labels=labels)))
    if clf_report:
        print(classification_report(y_test, y_pred, labels=labels, digits=3, zero_division=0))
    if conf_matrix:
        ConfusionMatrixDisplay.from_predictions(y_test, y_pred, labels=labels)
        plt.show()
    print("================================================================\n")

In [ ]:
eval_results_hmm(y_true_labels, y_pred_labels)

Weighted F1 of HMM = 0.2904
              precision    recall  f1-score   support

           1      0.361     0.389     0.375       714
           2      0.300     0.245     0.269       629
           3      0.149     0.453     0.225       309
           4      0.085     0.233     0.124       249
           5      0.255     0.595     0.357       482
           6      0.067     0.404     0.115        99

   micro avg      0.207     0.386     0.269      2482
   macro avg      0.203     0.387     0.244      2482
weighted avg      0.259     0.386     0.290      2482


